# AXA CHALLENGE Part I
**Jie ZHENG ** <br>
**Yunzhi BAI** <br>
**Xiaoxiao CHEN**
<br>
<br>
<br>
**This part is for the methode: train one model by using all the data.**<br>
<br>
**To run this part, please run it "step by step"! In the sections of "Training", there are many models provided, please just run one chosen model at one time, then run the submission section.** <br>
**For example: To use randomForest, just run the cells in the section "RandomForest". Then, do not run cells in other sections anymore!**
<br><br>
**IMPORTANT: Must provide the path (in the end of this notebook) to save the submisson file**
<br><br>
**IMPORTANT: There is the other part (train only one model over all assignments) in the directory.**

# 1. Preprocessing

## 1.1 Load Data

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

fields=['DATE','WEEK_END','DAY_WE_DS','TPER_TEAM','TPER_HOUR','SPLIT_COD','ASS_ASSIGNMENT',
        'CSPL_ACDCALLS','CSPL_ABNCALLS','CSPL_ABNCALLS1','CSPL_RECEIVED_CALLS']

train_fname = 'train_2011_2012_2013.csv'
df_train = pd.read_csv(train_fname, usecols=fields, sep=';')
df_train.head()

,DATE,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ASS_ASSIGNMENT,CSPL_ACDCALLS,CSPL_ABNCALLS,CSPL_ABNCALLS1,CSPL_RECEIVED_CALLS
0,2011-04-24 01:30:00.000,1,Dimanche,Nuit,1,855,Téléphonie,0,0,0,0
1,2011-04-24 01:30:00.000,1,Dimanche,Nuit,1,1587,Téléphonie,0,0,0,0
2,2011-04-24 01:30:00.000,1,Dimanche,Nuit,1,1589,Téléphonie,0,0,0,0
3,2011-04-24 01:30:00.000,1,Dimanche,Nuit,1,1591,Téléphonie,0,0,0,0
4,2011-04-24 01:30:00.000,1,Dimanche,Nuit,1,1555,Téléphonie,1,0,0,1


In [2]:
submission_fname = 'submission.txt'
dfsub= pd.read_csv(submission_fname, sep='\t')
dfsub = dfsub.drop('prediction',axis=1)
dfsub.head()

,DATE,ASS_ASSIGNMENT
0,2012-12-28 00:00:00.000,CMS
1,2012-12-28 00:00:00.000,Crises
2,2012-12-28 00:00:00.000,Domicile
3,2012-12-28 00:00:00.000,Gestion
4,2012-12-28 00:00:00.000,Gestion - Accueil Telephonique


## 1.2 Sum numbers of received calls for a given date and assignment

In [3]:
from pandas.core.frame import DataFrame
def sum_col_calls(dataframe):
    df = dataframe.copy()
    df = DataFrame({'sum_RECEIVED_CALLS':df.groupby(['DATE','WEEK_END','DAY_WE_DS','TPER_TEAM','TPER_HOUR',
                                                     'ASS_ASSIGNMENT'])['CSPL_RECEIVED_CALLS'].sum()}).reset_index()
    return df

In [4]:
train = sum_col_calls(df_train)

## 1.3 Define functions and dictionaries

In [5]:
options_weekdays = {'Monday' : 1,
            'Tuesday' : 2,
            'Wednesday' : 3,
            'Thursday' : 4,
            'Friday' : 5,
            'Saturday' : 6,
            'Sunday' : 7,
}


options_weekend = {'Monday' : 0,
            'Tuesday' : 0,
            'Wednesday' : 0,
            'Thursday' : 0,
            'Friday' : 0,
            'Saturday' : 1,
            'Sunday' : 1,
}

options_weekdays_fr = {'Lundi' : 1,
            'Mardi' : 2,
            'Mercredi' : 3,
            'Jeudi' : 4,
            'Vendredi' : 5,
            'Samedi' : 6,
            'Dimanche' : 7,
}

nuit_jours = {'Nuit':0,'Jours':1}

l = np.unique(train.ASS_ASSIGNMENT.values)
s = dict((l[i],i) for i in np.arange(len(l)))

In [6]:
import datetime

# gettime is for test_set, to construct features about date
def gettime(dataframe):
    df = dataframe.copy()
    df['Year'] = pd.DatetimeIndex(df['DATE']).year
    df['Month'] = pd.DatetimeIndex(df['DATE']).month
    df['Day'] = pd.DatetimeIndex(df['DATE']).day
    n = len(df)
    for i in np.arange(n):
        tmp = df.ix[i].DATE
        weekday = datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f').strftime('%A')
        hour = '{d.hour}'.format(d=datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f'))
        minute = datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f').strftime('%M')
        df.set_value(i, 'DAY_WE_DS', options_weekdays[weekday])
        df.set_value(i, 'WEEK_END', options_weekend[weekday])
        df.set_value(i, 'TPER_HOUR', hour+minute)
        df.set_value(i, 'ASS_ASSIGNMENT', s[df.ix[i].ASS_ASSIGNMENT])
        if (int(hour)>=8 and int(hour)<=22) or (int(hour)==7 and int(minute)==30) or (int(hour)==23 and int(minute)==0):
            df.set_value(i, 'TPER_TEAM', 1)
        else: df.set_value(i, 'TPER_TEAM', 0)
    return df

# getitime is for train_set, to construct features about date
def gettime_2(dataframe):
    df = dataframe.copy()
    df['Year'] = pd.DatetimeIndex(df['DATE']).year
    df['Month'] = pd.DatetimeIndex(df['DATE']).month
    df['Day'] = pd.DatetimeIndex(df['DATE']).day
    n = len(df)
    df = df.drop('TPER_HOUR',axis=1)
    
    for i in np.arange(n):
        df.set_value(i, 'DAY_WE_DS', options_weekdays_fr[df.ix[i].DAY_WE_DS])
        df.set_value(i, 'TPER_TEAM', nuit_jours[df.ix[i].TPER_TEAM])
        df.set_value(i, 'ASS_ASSIGNMENT', s[df.ix[i].ASS_ASSIGNMENT])
        
        tmp = df.ix[i].DATE
        weekday = datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f').strftime('%A')
        hour = '{d.hour}'.format(d=datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f'))
        minute = datetime.datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f').strftime('%M')
        df.set_value(i, 'TPER_HOUR', hour+minute)
    return df

        
# 07:30:00.000 Jours 
# 23:30:00.000 Nuit

In [7]:
from pandas.core.frame import DataFrame

# addMeanValues is to compute mean value over all assignments
def addMeanValues(df_train, df_test, byDAY_WE_DS=True, byDAY=True, byMonth=True, byHour=True, byTEAM=True, byYear=True):
    df = df_train.copy()
    df_2 = df_test.copy()
    if byDAY_WE_DS:
        df_meanByDAY_WE_DS = DataFrame({'meanByDAY_WE_DS':df.groupby(['DAY_WE_DS'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByDAY_WE_DS.set_index('DAY_WE_DS'),on='DAY_WE_DS')
        df_2 = df_2.join(df_meanByDAY_WE_DS.set_index('DAY_WE_DS'),on='DAY_WE_DS')
    if byDAY:
        df_meanByDAY = DataFrame({'meanByDay':df.groupby(['Day'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByDAY.set_index('Day'),on='Day')
        df_2 = df_2.join(df_meanByDAY.set_index('Day'),on='Day')
    if byMonth:
        df_meanByMONTH = DataFrame({'meanByMonth':df.groupby(['Month'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByMONTH.set_index('Month'),on='Month')
        df_2 = df_2.join(df_meanByMONTH.set_index('Month'),on='Month')
    if byHour:
        df_meanByHour = DataFrame({'meanByHour':df.groupby(['TPER_HOUR'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByHour.set_index('TPER_HOUR'),on='TPER_HOUR')
        df_2 = df_2.join(df_meanByHour.set_index('TPER_HOUR'),on='TPER_HOUR')
    if byTEAM:
        df_meanByTEAM = DataFrame({'meanByTEAM':df.groupby(['TPER_TEAM'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByTEAM.set_index('TPER_TEAM'),on='TPER_TEAM')
        df_2 = df_2.join(df_meanByTEAM.set_index('TPER_TEAM'),on='TPER_TEAM')
    if byYear:
        df_meanByYear = DataFrame({'meanByYear':df.groupby(['Year'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByYear.set_index('Year'),on='TPER_TEAM')
        df_2 = df_2.join(df_meanByYear.set_index('Year'),on='TPER_TEAM')     
    return df,df_2

# addMeanValues_2 is to compute mean value for each assignment
def addMeanValues_2(df_train, df_test, byDAY_WE_DS=True, byDAY=True, byMonth=True, byHour=True, byTEAM=True, byYear=True):
    df = df_train.copy()
    df_2 = df_test.copy()
    if byDAY_WE_DS:
        df_meanByDAY_WE_DS = DataFrame({'meanByDAY_WE_DS':df.groupby(['DAY_WE_DS','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByDAY_WE_DS.set_index(['DAY_WE_DS','ASS_ASSIGNMENT']),on=['DAY_WE_DS','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByDAY_WE_DS.set_index(['DAY_WE_DS','ASS_ASSIGNMENT']),on=['DAY_WE_DS','ASS_ASSIGNMENT'])
    if byDAY:
        df_meanByDAY = DataFrame({'meanByDay':df.groupby(['Day','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByDAY.set_index(['Day','ASS_ASSIGNMENT']),on=['Day','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByDAY.set_index(['Day','ASS_ASSIGNMENT']),on=['Day','ASS_ASSIGNMENT'])
    if byMonth:
        df_meanByMONTH = DataFrame({'meanByMonth':df.groupby(['Month','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByMONTH.set_index(['Month','ASS_ASSIGNMENT']),on=['Month','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByMONTH.set_index(['Month','ASS_ASSIGNMENT']),on=['Month','ASS_ASSIGNMENT'])
    if byHour:
        df_meanByHour = DataFrame({'meanByHour':df.groupby(['TPER_HOUR','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByHour.set_index(['TPER_HOUR','ASS_ASSIGNMENT']),on=['TPER_HOUR','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByHour.set_index(['TPER_HOUR','ASS_ASSIGNMENT']),on=['TPER_HOUR','ASS_ASSIGNMENT'])
    if byTEAM:
        df_meanByTEAM = DataFrame({'meanByTEAM':df.groupby(['TPER_TEAM','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByTEAM.set_index(['TPER_TEAM','ASS_ASSIGNMENT']),on=['TPER_TEAM','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByTEAM.set_index(['TPER_TEAM','ASS_ASSIGNMENT']),on=['TPER_TEAM','ASS_ASSIGNMENT'])
    if byYear:
        df_meanByYear = DataFrame({'meanByYear':df.groupby(['Year','ASS_ASSIGNMENT'])['sum_RECEIVED_CALLS'].mean()}).reset_index()
        df = df.join(df_meanByYear.set_index(['Year','ASS_ASSIGNMENT']),on=['Year','ASS_ASSIGNMENT'])
        df_2 = df_2.join(df_meanByYear.set_index(['Year','ASS_ASSIGNMENT']),on=['Year','ASS_ASSIGNMENT'])    
    return df,df_2

## 1.4 Construct features about time for train_set and test_set

In [8]:
dfsub_v1 = gettime(dfsub)
dfsub_v1.head(5)

,DATE,ASS_ASSIGNMENT,Year,Month,Day,DAY_WE_DS,WEEK_END,TPER_HOUR,TPER_TEAM
0,2012-12-28 00:00:00.000,1,2012,12,28,5.0,0.0,000,0.0
1,2012-12-28 00:00:00.000,2,2012,12,28,5.0,0.0,000,0.0
2,2012-12-28 00:00:00.000,3,2012,12,28,5.0,0.0,000,0.0
3,2012-12-28 00:00:00.000,5,2012,12,28,5.0,0.0,000,0.0
4,2012-12-28 00:00:00.000,6,2012,12,28,5.0,0.0,000,0.0


In [ ]:
train_v1 = gettime_2(train)
train_v1.head()

## 1.5 Add statistic information (average received calls)

In [8]:
train_v1, dfsub_v1 = addMeanValues_2(train_v1, dfsub_v1, byDAY_WE_DS=True, byDAY=True, byMonth=True, byHour=True, byTEAM=True, byYear=True)
train_v1.head()

,DATE,WEEK_END,DAY_WE_DS,TPER_TEAM,ASS_ASSIGNMENT,sum_RECEIVED_CALLS,Year,Month,Day,TPER_HOUR,meanByDAY_WE_DS,meanByDay,meanByMonth,meanByHour,meanByTEAM,meanByYear
0,2011-01-01 00:00:00.000,1,6,0,2,0,2011,1,1,0,0.000000,0.000000,0.001667,0.000000,0.000355,0.013558
1,2011-01-01 00:00:00.000,1,6,0,3,0,2011,1,1,0,2.225895,5.009535,7.572454,0.154762,0.089810,6.066290
2,2011-01-01 00:00:00.000,1,6,0,5,0,2011,1,1,0,0.000000,0.001199,0.004167,0.000000,0.000126,0.000000
3,2011-01-01 00:00:00.000,1,6,0,6,0,2011,1,1,0,0.000149,1.814748,2.751157,0.003074,0.000387,2.456820
4,2011-01-01 00:00:00.000,1,6,0,7,0,2011,1,1,0,0.000000,0.104213,0.162264,0.000000,0.000000,0.119052


In [9]:
dfsub_v1.head(5)

,DATE,ASS_ASSIGNMENT,Year,Month,Day,DAY_WE_DS,WEEK_END,TPER_HOUR,TPER_TEAM,meanByDAY_WE_DS,meanByDay,meanByMonth,meanByHour,meanByTEAM,meanByYear
0,2012-12-28 00:00:00.000,1,2012,12,28,5,0,0,0,0.010152,0.008547,0.014648,0.000000,0.000000,0.008778
1,2012-12-28 00:00:00.000,2,2012,12,28,5,0,0,0,0.010753,0.000000,0.001916,0.000000,0.000355,0.000822
2,2012-12-28 00:00:00.000,3,2012,12,28,5,0,0,0,8.279745,6.670141,6.525083,0.154762,0.089810,6.754066
3,2012-12-28 00:00:00.000,5,2012,12,28,5,0,0,0,0.002651,0.006748,0.006412,0.000000,0.000126,0.002271
4,2012-12-28 00:00:00.000,6,2012,12,28,5,0,0,0,3.423617,2.859543,2.338767,0.003074,0.000387,2.744470


## 1.6 Use OneHotEncoder to catagorical features

In [64]:
from sklearn.preprocessing import OneHotEncoder

train_v1_1 = train_v1[['WEEK_END','DAY_WE_DS','TPER_TEAM','TPER_HOUR','ASS_ASSIGNMENT','Year','Month','Day','meanByDAY_WE_DS','meanByDay','meanByMonth','meanByHour','meanByTEAM','meanByYear']].values
print np.shape(train_v1_1)
enc = OneHotEncoder(categorical_features=[False,True,False,True,True,True,True,True,False,False,False,False,False,False])
train_v2 = enc.fit_transform(train_v1_1)
print train_v2.toarray().shape

test_v1 = dfsub_v1[['WEEK_END','DAY_WE_DS','TPER_TEAM','TPER_HOUR','ASS_ASSIGNMENT','Year','Month','Day','meanByDAY_WE_DS','meanByDay','meanByMonth','meanByHour','meanByTEAM','meanByYear']].values
print np.shape(test_v1)
test_v2 = enc.transform(test_v1).toarray()
print test_v2.shape

(1030829, 14)
(1030829, 137)
(82909, 14)
(82909, 137)


# 2. Get the training and submission data

In [65]:
X_train = train_v2
y_train = train.sum_RECEIVED_CALLS.values.copy()
X_sub = test_v2

# 3. Training

## 3.1 RandomForest

In [66]:
from sklearn.ensemble import RandomForestRegressor
randomForest = RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, n_jobs=-1)   
randomForest.fit(X_train,y_train)
y_sub = randomForest.predict(X_sub)

In [29]:
np.array(y_sub).shape

(82909,)

# 4. Submission

In [67]:
sub = dfsub.copy()
sub.head()

,DATE,ASS_ASSIGNMENT
0,2012-12-28 00:00:00.000,CMS
1,2012-12-28 00:00:00.000,Crises
2,2012-12-28 00:00:00.000,Domicile
3,2012-12-28 00:00:00.000,Gestion
4,2012-12-28 00:00:00.000,Gestion - Accueil Telephonique


In [68]:
sub['prediction'] = y_sub
sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28 00:00:00.000,CMS,0.0
1,2012-12-28 00:00:00.000,Crises,0.0
2,2012-12-28 00:00:00.000,Domicile,0.1
3,2012-12-28 00:00:00.000,Gestion,0.0
4,2012-12-28 00:00:00.000,Gestion - Accueil Telephonique,0.0


**Must provide the path to save the submisson file**

In [69]:
path1 ='/home/minami_yunji/Challenges/AXA/result.txt'
sub.to_csv(path1, sep='\t',header=True, index=False)